## Joins with Quasi-stable Coloring

In [1]:
using Distributions
using DataStructures: counter, Dict, Set, Vector, inc!

In [2]:
n = 200000
numVertices = 100000
zipf = [1.0/(i^.5) for i in 1:numVertices]
zipf = zipf ./ sum(zipf)
nothing #hide

Let's generate two arrays of size $n$ representing the edges in a graph G where the edges are generated according to a zipf distribution. Then makes two graph formats for convenience.

In [3]:
d = DiscreteNonParametric(1:numVertices, zipf)
x1 = rand(d, n) .% numVertices
x2 = rand(d, n) .% numVertices
EDict = Dict()
for x in Set(x1)
    EDict[x] = Set()
end
for i in 1:length(x1)
    push!(EDict[x1[i]], x2[i])
end

numEdges = sum(length(EDict[x]) for x in keys(EDict))
ETable = Array{Int64}(undef, numEdges, 2)
edgeCounter = 1
for x in keys(EDict)
    for y in EDict[x]
        ETable[edgeCounter,1] = x
        ETable[edgeCounter,2] = y
        edgeCounter += 1
    end
end

hash them then count the hash values:

In [21]:
numPartitions = 8
inDegCounters = [counter(Int32) for _ in 1:numPartitions]
outDegCounters = [counter(Int32) for _ in 1:numPartitions]
hashETable = (ETable .% numPartitions) .+ 1
for i in 1:size(ETable)[1]
    inc!(inDegCounters[hashETable[i,1]], ETable[i,1])
    inc!(outDegCounters[hashETable[i,2]], ETable[i,2])
end

hashCardinality::Dict{Int, Dict{Int, Int}} = Dict()
hashMaxDeg::Dict{Int, Dict{Int, Int}} = Dict()
for c1 in 1:numPartitions
    hashMaxDeg[c1] = Dict()
    hashCardinality[c1] = Dict()
    for c2 in 1:numPartitions
        hashMaxDeg[c1][c2] = 0
        hashCardinality[c1][c2] = 0
        for x in keys(EDict)
            if x .% numPartitions + 1 == c1
                numEdgesToColor = 0
                for y in EDict[x]
                    if y .% numPartitions + 1 == c2
                        numEdgesToColor += 1
                    end
                end
                hashMaxDeg[c1][c2] = max(numEdgesToColor, hashMaxDeg[c1][c2])
                hashCardinality[c1][c2] += numEdgesToColor
            end
        end
    end
end

by *Walter's method* we get an upper cardinality bound of the following for 3-hop paths in our graph: 

In [22]:
estimate_prior = 0
for x in keys(hashMaxDeg)
    if x in keys(hashMaxDeg)
        for y in keys(hashMaxDeg[x])
            if y in keys(hashMaxDeg)
                for z in keys(hashMaxDeg[y])
                    if z in keys(hashMaxDeg)
                        for u in keys(hashMaxDeg[z])
                            estimate_prior += min(hashCardinality[x][y]*hashMaxDeg[y][z]*hashMaxDeg[z][u],
                                                hashMaxDeg[x][y]*hashCardinality[y][z]*hashMaxDeg[z][u],
                                                hashMaxDeg[x][y]*hashMaxDeg[y][z]*hashCardinality[z][u]
                                                )
                        end
                    end
                end
            end
        end
    end
end
estimate_prior

4364391525

## Graph Coloring
Now let's transform this into a graph coloring problem. We start by generating a lifted graph.

In [23]:
using Graphs
using QuasiStableColors
g = Graph(numVertices)
for x in keys(EDict)
    for y in EDict[x]
        add_edge!(g, x, y)
    end
end

minVal = 1000000
C = q_color(g, n_colors=numPartitions)
color_hash::Dict{Int, Int} = Dict()
for (color, nodes) in enumerate(C)
    for x in nodes
        color_hash[x-1] = color
    end
end

Next, we keep a variety of statistics about the edges between each pair of colors including the min, avg, and max degree as well as the total number of edges.

In [24]:
colorCardinality = Dict()
for c in 1:numPartitions
    colorCardinality[c] = 0
end 

colorColorCounter = Dict()
for x in keys(EDict)
    c1 = color_hash[x]
    colorCardinality[c1] += 1
    for y in EDict[x] 
        c2 = color_hash[y]
        if !(c1 in keys(colorColorCounter))
           colorColorCounter[c1] = Dict()
        end
        if !(c2 in keys(colorColorCounter[c1]))
           colorColorCounter[c1][c2] = counter(Int)
        end
        inc!(colorColorCounter[c1][c2], x)
    end
end

colorEdgeCardinality::Dict{Int, Dict{Int, Int}} = Dict()
colorEdgeMaxDeg::Dict{Int, Dict{Int, Int}} = Dict()
for c1 in keys(colorColorCounter)
    colorEdgeMaxDeg[c1] = Dict()
    colorEdgeCardinality[c1] = Dict()
    for c2 in keys(colorColorCounter[c1])
        colorEdgeMaxDeg[c1][c2] =  0 
        colorEdgeCardinality[c1][c2] = 0
        for v in values(colorColorCounter[c1][c2])
            colorEdgeMaxDeg[c1][c2] = max(v, colorEdgeMaxDeg[c1][c2])
            colorEdgeCardinality[c1][c2] += v
        end 
    end
end

colorEdgeCardinality::Dict{Int, Dict{Int, Int}} = Dict()
colorEdgeMinDeg::Dict{Int, Dict{Int, Int}} = Dict()
for c1 in keys(colorColorCounter)
    colorEdgeMinDeg[c1] = Dict()
    colorEdgeCardinality[c1] = Dict()
    for c2 in keys(colorColorCounter[c1])
        colorEdgeMinDeg[c1][c2] =  1000 
        colorEdgeCardinality[c1][c2] = 0
        if length(values(colorColorCounter[c1][c2])) < colorCardinality[c1]
            colorEdgeMinDeg[c1][c2] =  0 
        else
            for v in values(colorColorCounter[c1][c2])
                colorEdgeMinDeg[c1][c2] = min(v, colorEdgeMinDeg[c1][c2])
                colorEdgeCardinality[c1][c2] += v
            end 
        end
    end
end


colorEdgeCardinality::Dict{Int, Dict{Int, Int}} = Dict()
colorEdgeAvgDeg::Dict{Int, Dict{Int, Float64}} = Dict()
for c1 in keys(colorColorCounter)
    colorEdgeAvgDeg[c1] = Dict()
    colorEdgeCardinality[c1] = Dict()
    for c2 in keys(colorColorCounter[c1])
        colorEdgeAvgDeg[c1][c2] = colorCardinality[c1]
        colorEdgeCardinality[c1][c2] = 0
        for v in values(colorColorCounter[c1][c2])
            colorEdgeCardinality[c1][c2] += v
        end 
        colorEdgeAvgDeg[c1][c2] = float(colorEdgeCardinality[c1][c2])/colorEdgeAvgDeg[c1][c2]
    end
end

Lastly, we test the lower bounds, avg estimates, and upper bounds on the number of three-hop paths.

In [25]:
minEstimate = 0
count = 1
for x in keys(colorEdgeMinDeg)
    if count % 100 == 0
        print(count)
        print("\n")
    end
    count += 1
    for y in keys(colorEdgeMinDeg[x])
        if y in keys(colorEdgeMinDeg)
            for z in keys(colorEdgeMinDeg[y])
                if z in keys(colorEdgeMinDeg)
                    for u in keys(colorEdgeMinDeg[z])
                        minEstimate += colorEdgeCardinality[x][y]*colorEdgeMinDeg[y][z]*colorEdgeMinDeg[z][u]
                    end
                end
            end
        end
    end
end
minEstimate

0

In [26]:
avgEstimate = 0
count = 1
for x in keys(colorEdgeAvgDeg)
    for y in keys(colorEdgeAvgDeg[x])
        if y in keys(colorEdgeAvgDeg)
            for z in keys(colorEdgeAvgDeg[y])
                if z in keys(colorEdgeAvgDeg)
                    for u in keys(colorEdgeAvgDeg[z])
                        count += 1
                        avgEstimate += colorEdgeCardinality[x][y]*colorEdgeAvgDeg[y][z]*colorEdgeAvgDeg[z][u]
                    end
                end
            end
        end
    end
end
avgEstimate

5.941241909667022e6

In [27]:
maxEstimate = 0
count = 1
for x in keys(colorEdgeMaxDeg)
    if count % 100 == 0
        print(count)
        print("\n")
    end
    count += 1
    for y in keys(colorEdgeMaxDeg[x])
        if y in keys(colorEdgeMaxDeg)
            for z in keys(colorEdgeMaxDeg[y])
                if z in keys(colorEdgeMaxDeg)
                    for u in keys(colorEdgeMaxDeg[z]) # Note: The DSB using just a max degree statistic might be different than the polymatroid bound, and might be more accurate...
                        numVals1 = colorEdgeCardinality[x][y]/colorEdgeMaxDeg[x][y]
                        numVals2 = colorEdgeCardinality[x][y]/colorEdgeMaxDeg[x][y]
                        numVals3 = colorEdgeCardinality[x][y]/colorEdgeMaxDeg[x][y]
                        
                        
                        maxEstimate += min(colorEdgeCardinality[x][y]*colorEdgeMaxDeg[y][z]*colorEdgeMaxDeg[z][u],
                                        colorEdgeMaxDeg[x][y]*colorEdgeCardinality[y][z]*colorEdgeMaxDeg[z][u],
                                        colorEdgeMaxDeg[x][y]*colorEdgeMaxDeg[y][z]*colorEdgeCardinality[z][u]
                                        )
                    end
                end
            end
        end
    end
end
maxEstimate

114643832

In [28]:
count

9

The actual cardinality is:

In [29]:
cardinality = 0
for x in keys(EDict)
    for y in EDict[x]
        if y in keys(EDict)
            for z in EDict[y]
                if z in keys(EDict)
                    cardinality += length(EDict[z])
                end
            end
        end
    end
end
cardinality

7173717

In [30]:
print("Hashing relative error: $(estimate_prior / cardinality),\n color min relative error: $(minEstimate / cardinality),\n color avg relative error: $(avgEstimate / cardinality),\n color max relative error: $(maxEstimate / cardinality)")

Hashing relative error: 608.386353267072,
 color min relative error: 0.0,
 color avg relative error: 0.8281957470119077,
 color max relative error: 15.981092089358976

In [ ]:
597/15